## NLP. Классификация отзывов на фильмы. IMDB Dataset.

Произведем загрузку набора данных в DataFrame pandas, изначально разделив на тренировочный и испытательный наборы.

In [3]:
import pandas as pd
import os
import numpy as np
import re
basepath = 'C:/dataset/aclImdb'
labels = {'pos': 1, 'neg': 0}
df_train = pd.DataFrame()
df_test = pd.DataFrame()

for s in ('test', 'train'):
    for l in ('pos', 'neg'):
        path = os.path.join(basepath, s, l)
        for file in sorted(os.listdir(path)):
            with open(os.path.join(path, file), 
                      'r', encoding='utf-8') as infile:
                txt = infile.read()
            match = re.search(r'_(\d+)', file)
            rate = match.group(1)    
            if s == 'test':
                df_test = pd.concat([df_test, pd.DataFrame({'review': pd.Series(txt), 'rate':pd.Series(int(rate)-1), 'assessment': pd.Series(labels[l])})], ignore_index=True)
            else:
                df_train = pd.concat([df_train, pd.DataFrame({'review': pd.Series(txt), 'rate':pd.Series(int(rate)-1), 'assessment': pd.Series(labels[l])})], ignore_index=True) 

In [4]:
df_train.shape

(25000, 3)

Для задачи классификации необходимо, чтобы метки начинались с 0, поэтому мы вычли из каждого рейтинга единицу

In [6]:
df_train['rate'].unique()

array([8, 7, 9, 6, 2, 3, 0, 1], dtype=int64)

Для обучения модели перемешаем образцы, поскольку они отсортированы по assessment

In [8]:
df_train = df_train.reindex(np.random.permutation(df_train.index))
df_train.head()

,review,rate,assessment
6806,Actually one particular person/character isn't...,8,1
7136,Rarely do I see a film that I am totally engro...,8,1
5098,"The Farrelly brothers, Bobby and Peter, are at...",6,1
14635,Some illegal so-called asylum seeker comes to ...,0,0
23154,There really wasn't much of a story in this fi...,1,0


В основе нашей модели будет лежать логистическая регрессия, оптимальные гиперпараметры которой будут найдены с помощью решетчатого поиска. Вначале из текста убирается шум, а далее текст каждой рецензии будет преобразован в соответствии с моделью суммирования: создается словарь уникальных слов с отображением на число, а далее каждый отзыв преобразуется в вектор признаков, содержащий счетчик частоты появления каждого слова из словаря в каждом отзыве, а далее эти сырые частоты термов преобразуются в меры tf-idf. Все эти преобразования происходят с помощью функции TfidfVectorizer. Для удобства создадим pipeline.

In [10]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import GridSearchCV

import re

import re

def preprocessor(text):
    text = re.sub(r'<[^>]*>', '', text)  # Удаляем HTML-теги
    text = re.sub(r'[\W]+', ' ', text.lower())  # Заменяем не-алфавитные символы на пробелы
    return text


tfidf = TfidfVectorizer(strip_accents=None, lowercase=False, preprocessor=preprocessor, stop_words = 'english', ngram_range=(1, 1))

param_grid = [{'clf__penalty': ['l2'], 'clf__C': [0.1, 1, 10]}]
clf_pipe = Pipeline([('vect', tfidf), ('clf', LogisticRegression(random_state=0, max_iter=200))])

gs = GridSearchCV(clf_pipe, param_grid, scoring='accuracy', cv=5, verbose=2, n_jobs=-1)

X_train = df_train['review']
y_train = df_train['rate']
gs.fit(X_train, y_train)

print('Лучшие параметры:', gs.best_params_)
print('Правильность при перекрестной проверке %.3f' % gs.best_score_)

Fitting 5 folds for each of 3 candidates, totalling 15 fits
Лучшие параметры: {'clf__C': 1, 'clf__penalty': 'l2'}
Правильность при перекрестной проверке 0.425


In [11]:
clf = gs.best_estimator_

In [12]:
X_test, y_test = df_test['review'],  df_test['rate']

In [13]:
print('Правильность на испытательном наборе: %.3f' % clf.score(X_test,y_test)) 

Правильность на испытательном наборе: 0.418


Неплохо, учитывая, что оценка человека на фильм может легко варироваться в неком диапозоне. Проверим насколько наша модель правильно отличает плохой отзыв от хорошего:

In [15]:
predict_test = clf.predict(X_test)
predict_train = clf.predict(X_train)

In [16]:
assessment_test = [int(x) for x in (predict_test>=5) ]
assessment_train = [int(x) for x in (predict_train>=5) ]

In [ ]:
from sklearn.metrics import accuracy_score

Правильность при классификации на плохой и хороший отзыв.

In [47]:
print('Правильность на тренировочном наборе данных: ',accuracy_score(df_train['assessment'],assessment_train))

Правильность на испытательном наборе данных:  0.94212


In [45]:
print('Правильность на испытательном наборе данных:', accuracy_score(df_test['assessment'],assessment_test))

Правильность на тренировочном наборе данных: 0.86848


Можно было создать DL сеть с Embedding и LSTM слоями, но обучить такую модель было бы в разы сложнее, но не факт, что результат был бы намного лучше. 
Я думаю результат и так удовлетворителен. Целью ставилось создать модель и развернуть ее как сервис для оценки отзывов, что и было сделано. Если потре

При переходе на сайт необходимо подождать 1-2 минуты, пока он выйдет из "спящего" режима.

https://web-96q2.onrender.com/